In [1]:
import numpy as np

In [22]:
class Tensor(object):
    """docstring for tensor"""
    def __init__(self, initial_value,op,graph):
        self.initial_value = initial_value
        self.graph = graph
        self.op = op

    def __add__(self, other): # change the + operateor's methods, other is another self 
        return self.graph.add(self, other)
    def __radd__(self, other):
        return self.graph.add(other, self)

In [23]:
class BaseOp(object):
    """docstring for BaseOp"""
    def __init__(self, inputs,graph):
        self.inputs = [graph.convert(input_) for input_ in inputs]
        self.output = graph.tensor(op=self)
        self.graph  = graph

    def compute(self,sess,*args):
        raise NotImplementedError()
    def gradient(self,grad):
        raise NotImplementedError()
'''
class myParent( object ):
    def __init__( self, customParam ):
        self.parentNumber = 5
        self.customParam = customParam

class Child( myParent ):
    def __init__( self, customParam ):
        myParent.__init__( self, customParam )
        self.childNumber = 4        
'''        
        
class AddOp(BaseOp):
    def __init__(self, inputs,graph):
        BaseOp.__init__(self, inputs,graph)
        print("11111111111 is :"+str(inputs[0].initial_value))
        if inputs[0] is not None and inputs[1] is not None:
            self.output = graph.convert(   self.compute(inputs[0],inputs[1])   )
        #self.output = graph.convert(1000000000000000000000000000000)
    def compute(self,a=0,b=0):
        #a = self.inputs[0].initial_value
        #b = self.inputs[1].initial_value
        return a.initial_value + b.initial_value
    def gradient(self,grad):
        return [grad,grad]


In [24]:
class Graph(object):
	"""docstring for Graph"""
	def tensor(self,initial_value=None,op=None):
		return Tensor(initial_value=initial_value,op=op,graph=self)

	def convert(self,value):
		if isinstance(value,Tensor):
			return value
		return self.tensor(initial_value=value)

	def gradients(self,y,x_s):

		'''
		>>>queue.append(([1,1],1))
		>>>queue.pop(0)
		([1, 1], 1)



    	loss_op = graph.mean(graph.square(graph.transpose(y) - activations1))
    	parameters = [weights0, biases0, weights1, biases1]

    	gradients = graph.gradients(loss_op, parameters)
		'''
		queue = []
		queue.append((y,1))

		grads = {}

		while len(queue)>0:
			y,grad_y = queue.pop(0)
			grad_y = self.convert(grad_y) #convert grad_y to a tensor
			print(y.op)
			#gradients = y.op.gradient(grad_y)
		return gradients

	def add(self,a,b):
		op = AddOp([a,b],graph=self)
		return op.output

In [25]:
class Session(object):
    def __init__(self, graph):
        self.graph = graph
        self.state = {}
        
    def run_op(self, op, context):
        args = [self.eval_tensor(tensor, context) for tensor in op.inputs]
        return op.compute(self, *args)
    
    def eval_tensor(self, tensor, context):
        
        if tensor not in context:
            if tensor.op is not None:
                context[tensor] = self.run_op(tensor.op, context) #let context add the value in op.input
            elif tensor in self.state and self.state[tensor] is not None:
                context[tensor] = self.state[tensor]              #let context add the value in self.state
            elif tensor not in self.state and tensor.initial_value is not None:
                context[tensor] = self.state[tensor] = tensor.initial_value
                                                        #let context add the value by the initial_value of tensor
        return context[tensor]
    def run(self, tensors, feed_dict=None):
        
        context = {}

        if feed_dict:
            context.update(feed_dict)

        return [self.eval_tensor(tensor, context) for tensor in tensors]

In [26]:
graph = Graph()
a = graph.tensor(2)
b = graph.tensor(5)
c = a + b
print(c.initial_value)
print(c.op)
#grad_a, grad_b = graph.gradients(c, [a, b])

#sess = Session(graph)
#grad_a_, grad_b_ = sess.run([grad_a, grad_b], feed_dict={a: 2, b: 1})
#print(grad_a_)
#print(grad_b_)

11111111111 is :2
7
None
